In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dist
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphSAGE
from minisom import MiniSom  
import numpy as np
from torch_geometric.loader import NeighborLoader
from torch.utils.data import Dataset, DataLoader

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import umap
from torch_geometric.nn import GATConv

2025-02-02 16:28:40.676898: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738510120.703659 3769500 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738510120.711929 3769500 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-02 16:28:40.740944: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import sys
sys.path.append('/home/mei/nas/docker/thesis')
from dataloader.ts_reader import LSTMTSDataset,collate_fn
from dataloader.pyg_reader import GraphDataset

In [4]:
data_dir = "/home/mei/nas/docker/thesis/data/hdf/train"
config = {  
    "data_dir": "/home/mei/nas/docker/thesis/data/hdf",
    "graph_dir": "/home/mei/nas/docker/thesis/data/graphs",
    "mode": "k_closest",
    "k": 3
          
}

In [9]:
# === LSTM + Flat Dataset ===
lstm_dataset = LSTMTSDataset(data_dir)
lstm_loader = DataLoader(lstm_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# === Graph Dataset ===
# graph_loader = get_graph_dataloader(config, batch_size=32, shuffle=True)

graph_dataset = GraphDataset(config)
graph_data = graph_dataset.graph_data  #
graph_data.edge_index = graph_data.edge_index.contiguous() 

# 设定 `NeighborLoader` 进行采样
graph_loader = NeighborLoader(
    graph_data,
    num_neighbors=[10, 10],  # 2-hop 采样，每层 10 个邻居
    batch_size=32,
    shuffle=True
)

==> Loading precomputed graph from /home/mei/nas/docker/thesis/data/graphs/diagnosis_graph_k_closest_k3.pt
==> Loading flat features from /home/mei/nas/docker/thesis/data/hdf/final_flat.h5
x_flat shape: torch.Size([11698, 104]), num_nodes: 11698


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **模型**
model = MultiModalICUModel(
    flat_dim=104, ts_dim=163, hidden_dim=128, 
    gnn_dim=104, num_classes=2
).to(device)

# **优化器 & 损失**
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# **获取完整图数据（因为 GNN 处理整张图）**
graph_data = next(iter(graph_loader)).to(device)  # 只取一次完整的图

In [ ]:
for epoch in range(10):
    model.train()
    
    for batch in lstm_loader:
        flat, ts, seq_lengths, graph_data, labels, ts_target = batch
        flat, ts, labels, ts_target = flat.to(device), ts.to(device), labels.to(device), ts_target.to(device)

        optimizer.zero_grad()
        
        outputs, ts_pred = model(flat, ts, seq_lengths, graph_data)
        loss = loss_fn(outputs, labels, ts_pred, ts_target)
        
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch}, Loss: {loss.item()}")